In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

import math
import random

In [2]:
#0 - rock
#1 - paper
#2 - scissors

In [3]:
def get_score(left_move, right_move):
    delta = (
        right_move - left_move
        if (left_move + right_move) % 2 == 0
        else left_move - right_move
    )
    return 0 if delta == 0 else math.copysign(1, delta)

In [4]:
def rock_agent(observation, configuration):
    return 0

In [5]:
def paper_agent(observation, configuration):
    return 1

In [6]:
def scissors_agent(observation, configuration):
    return 2

In [7]:
def copy_opponent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return random.randrange(0, configuration.signs)

In [8]:
def random_choice(observation, configuration):
    return random.randrange(0, configuration.signs)

In [9]:
def rock_or_paper(observation, configuration):
    return random.randrange(0, 2)

In [10]:
def rock_or_scissors(observation, configuration):
    return random.choice([0, 2])

In [11]:
def paper_or_scissors(observation, configuration):
    return random.choice([1, 2])

In [12]:
last_react_action = None

def reactionary(observation, configuration):
    global last_react_action
    if observation.step == 0:
        last_react_action = random.randrange(0, configuration.signs)
    elif get_score(last_react_action, observation.lastOpponentAction) <= 1:
        last_react_action = (observation.lastOpponentAction + 1) % configuration.signs

    return last_react_action

In [13]:
last_counter_action = None


def counter_reactionary(observation, configuration):
    global last_counter_action
    if observation.step == 0:
        last_counter_action = random.randrange(0, configuration.signs)
    elif get_score(last_counter_action, observation.lastOpponentAction) == 1:
        last_counter_action = (last_counter_action + 2) % configuration.signs
    else:
        last_counter_action = (observation.lastOpponentAction + 1) % configuration.signs

    return last_counter_action

In [14]:
action_histogram = {}

def statistical(observation, configuration):
    global action_histogram
    if observation.step == 0:
        action_histogram = {}
        return
    action = observation.lastOpponentAction
    if action not in action_histogram:
        action_histogram[action] = 0
    action_histogram[action] += 1
    mode_action = None
    mode_action_count = None
    for k, v in action_histogram.items():
        if mode_action_count is None or v > mode_action_count:
            mode_action = k
            mode_action_count = v
            continue

    return (mode_action + 1) % configuration.signs

In [15]:
actions = []

def keep_winning(observation, configuration):
    global actions
    if observation.step == 0:
        answer = random.randrange(0, configuration.signs)
        actions.append(answer)
    elif get_score(actions[-1], observation.lastOpponentAction) == 1:
        answer = actions[-1]
        actions.append(answer)
    else:
        answer = random.randrange(0, configuration.signs)
        actions.append(answer)
    return answer

In [16]:
functions = [rock_agent, paper_agent, scissors_agent, copy_opponent, random_choice, rock_or_paper, rock_or_scissors, paper_or_scissors, reactionary, counter_reactionary, statistical, keep_winning]

In [17]:
d = {f.__name__: [0, 0, 0] for f in functions}

In [18]:
l = []
for i in range(len(functions)):
  for j in range(len(functions)):
    one_set = evaluate("rps", #environment to use - no need to change
                        [functions[i], functions[j]], #agents to evaluate
                        configuration={"episodeSteps": 200} #number of episodes
                      )
    print(functions[i].__name__, functions[j].__name__, end=': ')
    print(one_set)
    if one_set[0][0] > one_set[0][1]:
      d[functions[i].__name__][0] += 1
    elif one_set[0][0] < one_set[0][1]:
      d[functions[i].__name__][1] += 1
    else:
      d[functions[i].__name__][2] += 1

rock_agent rock_agent: [[0, 0]]
rock_agent paper_agent: [[-199.0, 199.0]]
rock_agent scissors_agent: [[199.0, -199.0]]
rock_agent copy_opponent: [[0, 0]]
rock_agent random_choice: [[0, 0]]
rock_agent rock_or_paper: [[-98.0, 98.0]]
rock_agent rock_or_scissors: [[107.0, -107.0]]
rock_agent paper_or_scissors: [[0, 0]]
rock_agent reactionary: [[-197.0, 197.0]]
rock_agent counter_reactionary: [[-100.0, 100.0]]
rock_agent statistical: [[-198.0, 198.0]]
rock_agent keep_winning: [[-198.0, 198.0]]
paper_agent rock_agent: [[199.0, -199.0]]
paper_agent paper_agent: [[0, 0]]
paper_agent scissors_agent: [[-199.0, 199.0]]
paper_agent copy_opponent: [[0, 0]]
paper_agent random_choice: [[0, 0]]
paper_agent rock_or_paper: [[95.0, -95.0]]
paper_agent rock_or_scissors: [[0, 0]]
paper_agent paper_or_scissors: [[-98.0, 98.0]]
paper_agent reactionary: [[-198.0, 198.0]]
paper_agent counter_reactionary: [[-99.0, 99.0]]
paper_agent statistical: [[-197.0, 197.0]]
paper_agent keep_winning: [[-199.0, 199.0]]
scis

In [19]:
df = pd.DataFrame.from_dict(d, orient='index', columns=['Wins', 'Losses', 'Draws'])
df = df.sort_values("Wins", ascending = False)
df

,Wins,Losses,Draws
reactionary,9,1,2
counter_reactionary,7,1,4
statistical,6,3,3
keep_winning,6,3,3
rock_agent,2,6,4
paper_agent,2,6,4
scissors_agent,2,6,4
rock_or_paper,2,6,4
rock_or_scissors,2,5,5
paper_or_scissors,2,5,5
